In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import os
from sklearn import tree, metrics
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import VotingRegressor
import matplotlib.pyplot as plt
from scipy.stats import norm
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import joblib

**Mounting from Google Drive**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
mplayers = pd.read_csv('/content/drive/My Drive/players/male_players (legacy).csv')

<ipython-input-3-e4ac2618f3a5>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  mplayers = pd.read_csv('/content/drive/My Drive/players/male_players (legacy).csv')


In [4]:
pd.set_option('display.max_columns', None)

In [5]:
mplayers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [6]:

mplayers.head()

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,value_eur,wage_eur,age,dob,height_cm,weight_kg,league_id,league_name,league_level,club_team_id,club_name,club_position,club_jersey_number,club_loaned_from,club_joined_date,club_contract_valid_until_year,nationality_id,nationality_name,nation_team_id,nation_position,nation_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,real_face,release_clause_eur,player_tags,player_traits,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,attacking_short_passing,attacking_volleys,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,100500000.0,550000.0,27,1987-06-24,169,67,53.0,La Liga,1.0,241.0,FC Barcelona,CF,10.0,NaN,2004-07-01,2018.0,52,Argentina,1369.0,CF,10.0,Left,3,4,5,Medium/Low,Normal (170-),Yes,NaN,"#Speedster, #Dribbler, #FK Specialist, #Acroba...","Finesse Shot, Speed Dribbler (AI), One Club Pl...",93.0,89.0,86.0,96.0,27.0,63.0,84,94,71,89,85,96,89,90,76,96,96,90,94,94,95,80,73,77,60,88,48,22,92,90,76,NaN,25,21,20,6,11,15,14,8,NaN,89+3,89+3,89+3,92+3,90+3,90+3,90+3,92+3,92+3,92+3,92+3,90+3,79+3,79+3,79+3,90+3,62+3,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,79000000.0,375000.0,29,1985-02-05,185,80,53.0,La Liga,1.0,243.0,Real Madrid CF,LW,7.0,NaN,2009-07-01,2018.0,38,Portugal,1354.0,LW,7.0,Right,4,5,5,High/Low,Normal (185+),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Power Free-Kick, Flair, Long Shot Taker (AI), ...",93.0,93.0,81.0,91.0,32.0,79.0,83,95,86,82,87,93,88,79,72,92,91,94,93,90,63,94,94,89,79,93,63,24,91,81,85,NaN,22,31,23,7,11,15,14,11,NaN,91+1,91+1,91+1,89+3,91+1,91+1,91+1,89+3,89+3,89+3,89+3,87+3,77+3,77+3,77+3,87+3,63+3,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,54500000.0,275000.0,30,1984-01-23,180,80,19.0,Bundesliga,1.0,21.0,FC Bayern München,SUB,10.0,NaN,2009-08-28,2017.0,34,Netherlands,105035.0,RS,11.0,Left,2,4,5,High/Low,Normal (170-185),Yes,NaN,"#Speedster, #Dribbler, #Distance Shooter, #Acr...","Diver, Injury Prone, Avoids Using Weaker Foot,...",93.0,86.0,83.0,92.0,32.0,64.0,80,85,50,86,86,93,85,83,76,90,93,93,93,89,91,86,61,78,65,90,47,39,89,84,80,NaN,29,26,26,10,8,11,5,15,NaN,84+3,84+3,84+3,88+2,87+3,87+3,87+3,88+2,88+2,88+2,88+2,87+3,78+3,78+3,78+3,87+3,64+3,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,52500000.0,275000.0,32,1981-10-03,195,95,16.0,Ligue 1,1.0,73.0,Paris Saint-Germain,ST,10.0,NaN,2012-07-01,2016.0,46,Sweden,1363.0,ST,10.0,Right,4,4,5,Medium/Low,Normal (185+),Yes,NaN,"#Poacher, #Aerial Threat, #Distance Shooter, #...","Power Free-Kick, Leadership, Flair, Long Shot ...",76.0,91.0,81.0,86.0,34.0,86.0,76,91,76,84

**Data preprocessing**

In [7]:
features_to_select = ['mentality_composure', 'attacking_volleys', 'goalkeeping_reflexes', 'goalkeeping_kicking',
                      'attacking_short_passing', 'dribbling', 'power_stamina', 'defending_standing_tackle',
                      'goalkeeping_positioning', 'movement_reactions', 'attacking_heading_accuracy',
                      'attacking_crossing', 'movement_sprint_speed', 'physic', 'goalkeeping_diving',
                      'defending_sliding_tackle', 'attacking_finishing', 'defending_marking_awareness',
                      'skill_ball_control', 'power_shot_power', 'potential', 'goalkeeping_handling',
                      'movement_balance', 'mentality_penalties', 'mentality_positioning', 'shooting', 'skill_fk_accuracy',
                      'mentality_aggression', 'age', 'power_strength', 'skill_curve',
                      'defending', 'movement_agility', 'height_cm', 'mentality_vision', 'skill_long_passing',
                      'goalkeeping_speed', 'mentality_interceptions', 'weight_kg', 'movement_acceleration', 'overall',
                      'skill_dribbling', 'pace', 'passing', 'power_long_shots']

corr_matrix = mplayers[features_to_select].corr()
corr_matrix['overall'].sort_values(ascending=False)

overall                        1.000000
movement_reactions             0.845753
potential                      0.695362
mentality_composure            0.683307
passing                        0.681114
dribbling                      0.606575
physic                         0.513279
attacking_short_passing        0.499757
mentality_vision               0.493282
skill_long_passing             0.483808
shooting                       0.482498
power_shot_power               0.480297
age                            0.458390
skill_ball_control             0.457011
skill_curve                    0.415863
power_long_shots               0.409086
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
goalkeeping_speed              0.385093
attacking_volleys              0.377048
skill_dribbling                0.369827
power_stamina                  0.360025
power_strength                 0.350786
mentality_positioning          0.350178


In [8]:
selected_cols = ['physic', 'goalkeeping_reflexes', 'goalkeeping_positioning', 'potential', 'shooting',
                 'goalkeeping_speed', 'attacking_volleys', 'mentality_interceptions', 'goalkeeping_handling',
                 'attacking_short_passing', 'mentality_vision', 'mentality_positioning', 'movement_reactions',
                 'goalkeeping_kicking', 'skill_fk_accuracy', 'mentality_composure', 'mentality_penalties',
                 'defending', 'skill_long_passing', 'attacking_crossing', 'mentality_aggression', 'goalkeeping_diving',
                 'age', 'power_long_shots', 'skill_curve', 'dribbling', 'attacking_finishing', 'skill_dribbling',
                 'movement_agility', 'power_shot_power', 'skill_ball_control', 'passing']

In [9]:
mplayers = pd.read_csv('/content/drive/My Drive/players/male_players (legacy).csv', usecols = selected_cols)
mplayers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 32 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   overall                  161583 non-null  int64  
 1   potential                161583 non-null  int64  
 2   age                      161583 non-null  int64  
 3   shooting                 143614 non-null  float64
 4   passing                  143614 non-null  float64
 5   dribbling                143614 non-null  float64
 6   defending                143614 non-null  float64
 7   physic                   143614 non-null  float64
 8   attacking_crossing       161583 non-null  int64  
 9   attacking_finishing      161583 non-null  int64  
 10  attacking_short_passing  161583 non-null  int64  
 11  attacking_volleys        161583 non-null  int64  
 12  skill_dribbling          161583 non-null  int64  
 13  skill_curve              161583 non-null  int64  
 14  skil

In [11]:
mplayers.fillna(0, inplace=True)

Feature Engineering

In [12]:
mplayers= mplayers.astype(int)
catching = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                          'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']
imputer = SimpleImputer(strategy='mean')
mplayers[catching] = imputer.fit_transform(mplayers[catching])
mplayers['catching'] = mplayers[catching].mean(axis=1)
mplayers.drop(columns=catching, inplace=True)


mentality = ['mentality_interceptions', 'mentality_composure', 'mentality_positioning',
                        'mentality_aggression',  'mentality_penalties', 'mentality_vision']
imputer = SimpleImputer(strategy='mean')
mplayers[mentality] = imputer.fit_transform(mplayers[mentality])
mplayers['mentality'] = mplayers[mentality].mean(axis=1)
mplayers.drop(columns=mentality, inplace=True)


ballskills = ['skill_dribbling', 'skill_long_passing', 'skill_curve', 'skill_fk_accuracy','skill_ball_control']
mplayers['ballskills'] = mplayers[ballskills].mean(axis=1)
mplayers.drop(columns=ballskills, inplace=True)

mplayers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 18 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   overall                  161583 non-null  int64  
 1   potential                161583 non-null  int64  
 2   age                      161583 non-null  int64  
 3   shooting                 161583 non-null  int64  
 4   passing                  161583 non-null  int64  
 5   dribbling                161583 non-null  int64  
 6   defending                161583 non-null  int64  
 7   physic                   161583 non-null  int64  
 8   attacking_crossing       161583 non-null  int64  
 9   attacking_finishing      161583 non-null  int64  
 10  attacking_short_passing  161583 non-null  int64  
 11  attacking_volleys        161583 non-null  int64  
 12  movement_reactions       161583 non-null  int64  
 13  power_shot_power         161583 non-null  int64  
 14  powe

In [13]:
shooting_cols = ['shooting', 'power_long_shots', 'power_shot_power', 'attacking_finishing', 'attacking_volleys']
mplayers['shooting_avg'] = mplayers[shooting_cols].mean(axis=1)
mplayers.drop(columns=shooting_cols, inplace=True)
mplayers.rename(columns={'shooting_avg': 'shooting'}, inplace=True)

passing_cols = ['passing', 'attacking_crossing', 'attacking_short_passing', 'movement_reactions']
mplayers['passing_avg'] = mplayers[passing_cols].mean(axis=1)
mplayers.drop(columns=passing_cols, inplace=True)
mplayers.rename(columns={'passing_avg': 'passing'}, inplace=True)

mplayers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   overall     161583 non-null  int64  
 1   potential   161583 non-null  int64  
 2   age         161583 non-null  int64  
 3   dribbling   161583 non-null  int64  
 4   defending   161583 non-null  int64  
 5   physic      161583 non-null  int64  
 6   catching    161583 non-null  float64
 7   mentality   161583 non-null  float64
 8   ballskills  161583 non-null  float64
 9   shooting    161583 non-null  float64
 10  passing     161583 non-null  float64
dtypes: float64(5), int64(6)
memory usage: 13.6 MB


Training


In [14]:
y = mplayers['overall']
x = mplayers.drop('overall', axis=1)

In [15]:
sc = StandardScaler()
scaled = sc.fit_transform(x)

In [16]:
x=pd.DataFrame(scaled, columns=x.columns)

In [18]:
xtrain, xtest, Ytrain, Ytest = train_test_split(x, y, test_size= 0.1, random_state=42, stratify=y)

(145424, 10)

In [ ]:
xtrain.shape

Training with XGB_REGRESSOR

In [19]:
xgb = XGBRegressor()

In [20]:
xgb.fit(xtrain, Ytrain)

xgb_ypred = xgb.predict(xtest)
xgbabsolute_err = mean_absolute_error(xgb_ypred, Ytest)
xgbabsolute_err

0.7133499982149134

Training with RANDOM FOREST REGRESSOR

In [21]:
rfg=RandomForestRegressor()

In [22]:
rfg.fit(xtrain, Ytrain)
rfg_ypred =rfg.predict(xtest)
rfg_ypred

array([57.66, 67.  , 66.69, ..., 58.75, 52.83, 74.78])

In [23]:
rfgabsolute_err = mean_absolute_error(rfg_ypred,Ytest)
rfgabsolute_err

0.6551401695649485

Training with GRADIENT BOOST REGRESSOR

In [24]:
gbr = GradientBoostingRegressor()

In [25]:
gbr.fit(xtrain, Ytrain)
gbr_ypred = gbr.predict(xtest)

In [26]:
gbrabsolute_err = mean_absolute_error(gbr_ypred, Ytest)
gbrabsolute_err

0.9464909711286751

Cross-Validation

In [27]:
# The hyperparameters for Random Forest Regressor and for Gradient Boost Regressor are:

parameters_rfg = {
    'n_estimators': [10, 15, 20],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 6, 10]
}
parameters_gbr = {
    'n_estimators': [10, 15, 20],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01, 0.001]
}

In [28]:
rfg_gridsearch = GridSearchCV(estimator = RandomForestRegressor(), param_grid = parameters_rfg, scoring = 'neg_mean_absolute_error', cv=5)
rfg_gridsearch.fit(xtrain, Ytrain)
best_rfg = rfg_gridsearch.best_estimator_

gbr_gridsearch = GridSearchCV(estimator = GradientBoostingRegressor(), param_grid = parameters_gbr, scoring = 'neg_mean_absolute_error', cv=5)
gbr_gridsearch.fit(xtrain, Ytrain)
best_gbr = gbr_gridsearch.best_estimator_

In [30]:
bestrfg_ypred = best_rfg.predict(xtest)
best_rfg_abserr = mean_absolute_error(bestrfg_ypred, Ytest)
print("The mean absolute error for the best Random Forest Model is: ", best_rfg_abserr)

bestgbr_ypred = best_gbr.predict(xtest)
best_gbr_abserr = mean_absolute_error(bestgbr_ypred, Ytest)
print("The mean absolute error for the best Gradient Boost Regressor is: ", best_gbr_abserr)

The mean absolute error for the best Random Forest Model is:  0.6710375120967335
The mean absolute error for the best Gradient Boost Regressor is:  1.1312012131511435


The ENSEMBLE MODEL using the Random Forest Regressor and Gradient Boost Regressor models above

In [31]:
ensemblemodel = VotingRegressor(estimators=[('rfg', rfg), ('gbr', gbr)])

ensemblemodel.fit(xtrain, Ytrain)
ensemblepredictions = ensemblemodel.predict(xtest)

In [33]:
ensemble_abserr = mean_absolute_error(ensemblepredictions, Ytest)
ensemblepredictions
xtest
Ytest

print("Mean absolute error for Random Forest: ", mean_absolute_error(rfg_ypred, Ytest))
print("Mean absolute error for Gradient Boost Regressor: ", best_gbr_abserr)
print("Mean absolute error for Ensemble Model: ", ensemble_abserr)

Mean absolute error for Random Forest:  0.6551401695649485
Mean absolute error for Gradient Boost Regressor:  1.1312012131511435
Mean absolute error for Ensemble Model:  0.7554320600694819


In [34]:
stdeviation = np.std(ensemblepredictions)
meanprediction = np.mean(ensemblepredictions)
print("The standard deviation is:", stdeviation)
print("The mean prediction is:", meanprediction)

In [37]:
# creation of the confidence level and margin of error
conf_level = 0.95
z_score = norm.ppf((1 + conf_level)/2)
margin_error = z_score *(stdeviation/np.sqrt(len(ensemblepredictions)))

ubound = meanprediction + margin_error
lbound = meanprediction - margin_error
conf_percentage = ((1-2 * (1 - conf_level)) * 100)

print("The lower bound for the regressors is:", lbound)
print("The upper bound for the regressors is:", ubound)
print("The margin of error for the regressors:", margin_error)
print("The confidence percentage for the regressors is:", conf_percentage)

The lower bound: 65.57527720075282
The upper bound: 65.78865538871077
The margin of error: 0.10668909397897354
The confidence percentage: 89.99999999999999


Training with the new Dataset(players22)


In [38]:
players2 = pd.read_csv('/content/drive/My Drive/players/players_22-1.csv', usecols =selected_cols)

In [39]:
players2.fillna(0, inplace=True)

In [40]:
players2= players2.astype(int)

catching = ['goalkeeping_diving', 'goalkeeping_handling', 'goalkeeping_kicking',
                          'goalkeeping_positioning', 'goalkeeping_reflexes', 'goalkeeping_speed']
imputer = SimpleImputer(strategy='mean')
players2[catching] = imputer.fit_transform(players2[catching])
players2['goalkeeping'] = players2[catching].mean(axis=1)
players2.drop(columns=catching, inplace=True)


mentality = ['mentality_interceptions', 'mentality_composure', 'mentality_positioning',
                        'mentality_aggression',  'mentality_penalties', 'mentality_vision']
imputer = SimpleImputer(strategy='mean')
players2[mentality] = imputer.fit_transform(players2[mentality])
players2['mentality_att'] = players2[mentality].mean(axis=1)
players2.drop(columns=mentality, inplace=True)


ballskills = ['skill_dribbling', 'skill_long_passing', 'skill_curve', 'skill_fk_accuracy','skill_ball_control']
players2['goodballskills'] = players2[ballskills].mean(axis=1)
players2.drop(columns=ballskills, inplace=True)

shooting_cols = ['shooting', 'power_long_shots', 'power_shot_power', 'attacking_finishing', 'attacking_volleys']
players2['shooting_avg2'] = players2[shooting_cols].mean(axis=1)
players2.drop(columns=shooting_cols, inplace=True)
players2.rename(columns={'shooting_avg2': 'shooting'}, inplace=True)

passing_cols = ['passing', 'attacking_crossing', 'attacking_short_passing', 'movement_reactions']
players2['passing_avg2'] = players2[passing_cols].mean(axis=1)
players2.drop(columns=passing_cols, inplace=True)
players2.rename(columns={'passing_avg2': 'passing'}, inplace=True)

players2.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19239 entries, 0 to 19238
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   overall                  19239 non-null  int64  
 1   potential                19239 non-null  int64  
 2   age                      19239 non-null  int64  
 3   shooting                 19239 non-null  int64  
 4   passing                  19239 non-null  int64  
 5   dribbling                19239 non-null  int64  
 6   defending                19239 non-null  int64  
 7   physic                   19239 non-null  int64  
 8   attacking_crossing       19239 non-null  int64  
 9   attacking_finishing      19239 non-null  int64  
 10  attacking_short_passing  19239 non-null  int64  
 11  attacking_volleys        19239 non-null  int64  
 12  movement_reactions       19239 non-null  int64  
 13  power_shot_power         19239 non-null  int64  
 14  power_long_shots      

In [42]:
y1 = players2['overall']
x1 = players2.drop('overall', axis=1)
x1

,potential,age,dribbling,defending,physic,goalkeeping,mentality_att,goodballskills,shooting,passing
0,93,34,95,34,65,9.000000,73.833333,94.0,91.0,90.25
1,92,32,86,44,82,8.500000,80.666667,81.4,90.6,82.00
2,91,36,88,34,75,9.666667,74.333333,83.6,92.4,85.25
3,91,29,94,37,63,9.833333,77.000000,89.2,82.6,86.50
4,91,30,88,64,78,9.333333,82.666667,88.0,86.4,93.00
...,...,...,...,...,...,...,...,...,...,...
19234,52,22,48,42,49,8.166667,42.333333,42.6,34.6,48.75
19235,59,19,46,41,51,7.833333,46.500000,43.4,38.2,51.00
19236,55,21,49,41,52,6.166667,42.500000,43.8,38.0,44.75
19237,60,19,48,15,42,8.833333,38.500000,38.0,43.6,37.75


In [43]:
x1scaled = sc.fit_transform(x1)
x1 = pd.DataFrame(x1scaled, columns=x1.columns)
x1train,x1test,Y1train,Y1test=train_test_split(x1,y1,test_size=0.1,random_state=42)

In [48]:
rfg.fit(x1train, Y1train)
rfg_ypred2 =rfg.predict(x1test)
rfg_ypred2

array([63.41, 57.  , 53.61, ..., 65.92, 75.72, 68.74])

In [49]:
rfgabsolute_err2 = mean_absolute_error(rfg_ypred2,Y1test)
rfgabsolute_err2

0.7011798336798337

In [50]:
gbr.fit(x1train, Y1train)
gbr_ypred2 = gbr.predict(x1test)

In [51]:
gbrabsolute_err2 = mean_absolute_error(gbr_ypred2, Y1test)
gbrabsolute_err2

0.9452194537978091

  Hyperparameter tuning

In [53]:
gridparam = {
    'n_estimators': [10, 15, 20],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

gsearch = GridSearchCV(estimator=rfg, param_grid=gridparam, cv=5, n_jobs=-1, verbose=2)
gsearch.fit(x1train, Y1train)
best_param = gsearch.best_params_

best_rfg = RandomForestRegressor(**best_param)
best_rfg.fit(x1train, Y1train)
rfg_tunedpredy = best_rfg.predict(x1test)
rfg_tuned_abserr = mean_absolute_error(rfg_tunedpredy, Y1test)
print("Best Hyperparameters:", best_param)
print("Mean Absolute Error for Tuned Random Forest: ", rfg_tuned_abserr)

Fitting 5 folds for each of 81 candidates, totalling 405 fits
Best Hyperparameters: {'max_depth': 20, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 20}
Mean Absolute Error for Tuned Random Forest:  0.7241834481357148


In [55]:
gridparam2 = {
    'n_estimators': [10, 15, 20],
    'max_depth': [3, 5, 9],
    'learning_rate': [0.01, 0.1, 0.2],
    'subsample': [0.8, 1.0],
}

gsearch2 = GridSearchCV(estimator= gbr, param_grid =gridparam2, cv=5, n_jobs=-1, verbose=2)
gsearch2.fit(x1train, Y1train)
best_param2 = gsearch2.best_params_

best_gbr = GradientBoostingRegressor(**best_param2)
best_gbr.fit(x1train, Y1train)
gbr_tunedpredy = best_gbr.predict(x1test)
gbr_tuned_abserr = mean_absolute_error(gbr_tunedpredy, Y1test)
print("Best Hyperparameters:", best_param2)
print("Mean Absolute Error for Tuned Gradient Boosting Regressor: ", gbr_tuned_abserr)

Fitting 5 folds for each of 54 candidates, totalling 270 fits
Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 9, 'n_estimators': 20, 'subsample': 1.0}
Mean Absolute Error for Tuned Gradient Boosting Regressor:  0.7564697136145009


In [56]:
ensemblemodel = VotingRegressor(estimators=[('rfg', rfg), ('gbr', gbr)])
ensemblemodel.fit(xtrain, Ytrain)
ensemblepredictions = ensemblemodel.predict(xtest)
print(ensemblepredictions)
print(xtest)
print(Ytest)

ensemble_abserr2 = mean_absolute_error(ensemblepredictions, Ytest)
ensemble_abserr2

[57.71871136 66.49514411 67.11502105 ... 58.46958064 52.4965031
 74.15822429]
        potential       age  dribbling  defending    physic  catching  \
30172   -0.438403 -1.096996   0.267060  -1.266488  0.151836 -0.403432   
41228   -0.598170  0.615996   0.312930  -0.821278  0.286070 -0.331435   
75100   -0.438403  0.401872   0.771638  -0.509631  0.688771 -0.341720   
63911    1.159271 -1.525244   0.221189  -0.999362 -0.340355 -0.382861   
131921   0.679969 -0.454624   0.450542   0.825998  0.330814 -0.362291   
...           ...       ...        ...        ...       ...       ...   
158534   0.360434 -1.096996   0.496413  -0.687715 -0.832546 -0.362291   
103412   0.040899 -1.096996  -0.054035   0.202704 -0.250866 -0.352005   
83895   -0.438403 -0.882872  -0.925579   0.692435  0.107091 -0.331435   
32385   -0.757937 -1.525244  -0.008165  -1.177446 -0.295611 -0.475428   
34784    0.999503 -0.026376   1.138603   0.870519  0.778261 -0.465143   

        mentality  ballskills  shooting   pas

0.7538618825583244

Copying data to a file

In [57]:
scfile_path = '/content/drive/My Drive/players/scaler_ensemble.pkl'
joblib.dump(sc, scfile_path)

['/content/drive/My Drive/players/scaler_ensemble.pkl']

In [58]:
import pickle
spfile = '/content/drive/My Drive/Colab Notebooks/sports_prediction_ensemble_model.pkl'
pickle.dump(ensemblemodel, open(spfile, 'wb'))
loadedm = pickle.load(open(spfile, 'rb'))